## This notebook will load some existing GEOJSON files and show them with Allen_MERFISH cells in a Napari window

In [1]:
%gui qt
import napari
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from shapely.geometry import Polygon, Point,LineString,  mapping


In [2]:
import json
import geojson


## Make a napari viewer...

In [3]:
v = napari.Viewer()

## Load a cell x gene table, here I'm using the old one

In [13]:
# load one cellxgene table
all_cells = pd.read_csv("/Users/brian/Downloads/fixed_1001844875(2).csv")


In [15]:
cell_plots = v.add_points(all_cells[["x_um","y_um"]], symbol='disc',
                                 name = "old segmentation",edge_color= [0,0,0,0], size = 10.5,
                                 face_color= 'b' , blending = "translucent", opacity = 0.8)

In [18]:
with open("/Users/brian/Allen_MERFISH_annotations_geo.json",'r') as r:
    shapejson = geojson.load(r)
    
#last element in the geojson file is the column axis, so ignore it for now
layer_annotations = shapejson["geometries"][:-1]

In [19]:
for geo in layer_annotations:
    shape_coordinates = np.array(geo["coordinates"][0])
    v.add_shapes([shape_coordinates] ,
        shape_type='polygon',
        edge_width=1,
        edge_color='coral',
        face_color='royalblue',
        name=geo["name"])

## use `shapely` to assign regions to cells

In [23]:
cell_is_in_layer = {p["name"]:[Polygon(p["coordinates"][0]).intersects(Point(a))  for a  in all_cells[["x_um","y_um"]].values] for ii,p in enumerate(layer_annotations)}

In [24]:
all_cells["layer"] = "outside_VISp"
for k in cell_is_in_layer.keys():
    all_cells.loc[cell_is_in_layer[k],["layer"]] = k


In [25]:
all_cells

,Unnamed: 0,Unnamed: 0.1,1700022I11Rik,1810046K07Rik,5031425F14Rik,5730522E02Rik,Acta2,Adam2,Adamts2,Adamts4,...,Wipf3,Wnt7b,Zfp804b,area,position,total_reads,simpleClass,x_um,y_um,layer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,170.10,[1161.11337076 6178.3830543 ],870,67,1161.113371,6178.383054,outside_VISp
1,1,1,0,0,1,1,0,0,0,0,...,1,0,1,206.55,[1162.91337076 6113.1330543 ],454,233,1162.913371,6113.133054,outside_VISp
2,2,2,0,0,0,1,0,0,0,0,...,2,0,0,184.68,[1168.31337076 6046.0830543 ],269,97,1168.313371,6046.083054,outside_VISp
3,3,3,0,0,0,0,2,0,0,0,...,0,0,0,87.48,[1163.81337076 6202.6830543 ],340,105,1163.813371,6202.683054,outside_VISp
4,4,4,0,0,0,1,0,0,0,2,...,3,0,1,155.52,[1166.96337076 6083.8830543 ],241,72,1166.963371,6083.883054,outside_VISp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,2394,2394,0,0,0,0,3,1,0,0,...,2,0,0,277.02,[3144.26337076 5201.4330543 ],1062,96,3144.263371,5201.433054,outside_VISp
2395,2395,2395,0,0,0,0,6,1,0,1,...,3,1,1,200.07,[3154.61337076 5331.0330543 ],188,0,3154.613371,5331.033054,outside_VISp
2396,2396,2396,0,1,0,1,3,2,0,3,...,5,0,0,338.58,[3165.41337076 5190.1830543 ],1322,99,3165.413371,5190.183054,outside_VISp
2397,2397,2397,1,0,0,0,0,0,0,2,...,0,0,0,189.54,[3189.71337076 5292.7830543 ],120,96,3189.713371,5292.783054,outside_VISp


In [ ]:
#